In [ ]:
! pip install langchain faiss-cpu sentence-transformers google-generativeai langchain-community python-dotenv langchain-openai

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import pickle
import os

# Read the entire file as a single string
file_path = "/content/bugreport-lynx-BP2A.250605.031.A2-2025-06-24-13-51-27.txt"
with open(file_path, "r", encoding="latin-1") as file:
    full_text = file.read()

# Initialize the splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=5000,          # Adjust as needed
    chunk_overlap=500,        # Adjust based on how much context you want retained
    separators=["\n\n", "\n", ".", " "]  # Split by paragraphs, then lines, then sentences
)

# Create document chunks
split_texts = text_splitter.split_text(full_text)
document_based_chunks = [Document(page_content=chunk, metadata={}) for chunk in split_texts]

# Print number of chunks and optionally one sample
print(f"Number of chunks: {len(document_based_chunks)}")
print(f"First chunk preview:\n{document_based_chunks[0].page_content[:500]}")

# Populate FAISS
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
vector_store = FAISS.from_documents(document_based_chunks, embedding_model)
# Save FAISS index and document mapping
vector_store.save_local("faiss_index_adb")
vector_store_adb = FAISS.load_local("faiss_index_adb", embedding_model, allow_dangerous_deserialization = True)